In [1]:
import sys

sys.path.append("..")
%load_ext autoreload
%autoreload 2
from trl_wrapper.trainer_wrapper import TrainerWrapper, GRPO_WRITING_CONFIG, SMOL_LM_135M

cfg = GRPO_WRITING_CONFIG
cfg.train_batch_size = 2
cfg.num_generations = 2
cfg.model_id_or_path = SMOL_LM_135M
cfg.max_samples = 10

/Users/brianfitzgerald/Documents/GitHub/smolmodels/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


INFO 04-13 09:28:06 [__init__.py:239] Automatically detected platform cpu.


2025-04-13 09:28:07,570	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


In [2]:
wrapper = TrainerWrapper(cfg, True)
wrapper.init_model()

2025-04-13 09:28:09.256 | INFO     | trl_wrapper.trainer_wrapper:__init__:297 - Setting padding side to: right
2025-04-13 09:28:09.268 | INFO     | trl_wrapper.trainer_wrapper:init_model:312 - Using device: mps
2025-04-13 09:28:09.268 | INFO     | trl_wrapper.trainer_wrapper:init_model:326 - Loading model HuggingFaceTB/SmolLM2-135M-Instruct with attn_impl: sdpa


In [8]:
wrapper.init_data_module()

2025-04-13 09:38:54.653 | INFO     | trl_wrapper.trainer_wrapper:init_data_module:352 - Using chat template override: smollmv2
2025-04-13 09:38:54.654 | INFO     | trl_wrapper.wrapper_config:__init__:144 - Cache dir: ../dataset_caches/writing_g_r_p_o_data_module
Map: 100%|██████████| 1184/1184 [00:00<00:00, 27392.88 examples/s]


In [4]:
wrapper.data_module.train_dataset

Dataset({
    features: ['instruction', 'paragraph', 'text', 'title', 'author', 'id', 'prompt'],
    num_rows: 10647
})

In [5]:
wrapper.init_trainer()

2025-04-13 09:28:11.005 | INFO     | trl_wrapper.trainer_wrapper:init_trainer:390 - Saving output to: ./runs/04-13-9-28-669960-smollm2-135m-instruct-
2025-04-13 09:28:11.007 | INFO     | trl_wrapper.trainer_wrapper:init_trainer:413 - Initializing trainer, run_name: 04-13-9-28-669960-smollm2-135m-instruct-, wandb project: writing-grpo
2025-04-13 09:28:11.007 | INFO     | trl_wrapper.trainer_wrapper:init_trainer:496 - Using vllm: False


In [ ]:
wrapper.train()

2025-04-13 09:41:36.145 | INFO     | trl_wrapper.trainer_wrapper:train:659 - Starting training.


Scores: []
Score sums: []


RuntimeError: The expanded size of the tensor (2) must match the existing size (0) at non-singleton dimension 0.  Target sizes: [2].  Tensor sizes: [0]

2025-04-13 09:41:59.972 | INFO     | synthetic_data.tasks.writing:score_writing:411 - Judging 2 completions with gemini-2.0-flash


Result: [{'Adherence to Instructions': 8.0, 'Believable Character Actions': 7.0, 'Nuanced Characters': 6.0, 'Consistent Voice/Tone of Writing': 7.0, 'Imagery and Descriptive Quality': 7.0, 'Elegant Prose': 7.0, 'Emotionally Engaging': 6.0, 'Emotionally Complex': 5.0, 'Coherent': 12.0, 'Meandering': 8.0, 'Weak Dialogue': 4.0, "Tell-Don't-Show": 4.0, 'Unsurprising or Uncreative': 5.0, 'Amateurish': 5.0, 'Purple Prose': 3.0, 'Overwrought': 4.0, 'Incongruent Ending Positivity': 2.0, 'Unearned Transformations': 2.0, 'Well-earned Lightness or Darkness': 7.0, 'Sentences Flow Naturally': 11.0, 'Overall Reader Engagement': 6.0, 'Overall Impression': 6.0}, {'Adherence to Instructions': 3.0, 'Believable Character Actions': 4.0, 'Nuanced Characters': 3.0, 'Consistent Voice/Tone of Writing': 8.0, 'Imagery and Descriptive Quality': 5.0, 'Elegant Prose': 4.0, 'Emotionally Engaging': 2.0, 'Emotionally Complex': 2.0, 'Coherent': 3.0, 'Meandering': 15.0, 'Weak Dialogue': 1.0, "Tell-Don't-Show": 16.0, 'U